In [159]:
import pandas as pd
from pandasql import sqldf

data = pd.read_csv("raw_event_log.csv")
new_dataframe = pd.DataFrame(data)

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT t2.user_id AS User_id, 
        t2.timestamp AS Date_of_Activation, 
        t2.total_logins AS Total_logins, 
        t2.total_vault AS Total_vault_events, 
        ROUND((JULIANDAY(t2.timestamp) - JULIANDAY(d4.timestamp)) * 86400) as Difference_in_seconds
        FROM
        (
        SELECT t.user_id, t.timestamp, t.total_logins, count(d3.event) as total_vault
        FROM
        (
        SELECT DISTINCT d1.user_id, d1.timestamp, count(d2.event) as total_logins
        FROM new_dataframe d1
        left join new_dataframe d2
        on d2.user_id = d1.user_id
        and d2.event = "login"
        where d1.event = "activate"
        group by d1.user_id
        ) as t
        left join new_dataframe d3
        on d3.user_id = t.user_id
        and d3.event = "add_to_vault"
        group by t.user_id
        ) as t2
        inner join new_dataframe d4
        on t2.user_id = d4.user_id
        where d4.event = "create"
        group by t2.user_id
   ;""" 

    
result= pysqldf(q)
result

result.to_csv("Dataset1.csv", encoding='utf-8', index=False)


,User_id,Date_of_Activation,Total_logins,Total_vault_events,Difference_in_seconds
0,0001fa9e7f7ad2df22d89a8766f5d644,2020-01-23 04:06:56,4,14,18245.0
1,000531dfe2623f4920ef8b195e73a0df,2020-03-08 14:39:30,8,26,8435.0
2,000555516893e0ebc990713f280275f1,2020-01-17 04:52:12,0,2,786.0
3,000fbde3e89cd60bcff2fafbd7941801,2020-01-26 03:26:36,8,20,21188.0
4,00101f952217a06f678f50f1c85c86cc,2020-01-28 04:35:10,8,27,11207.0
...,...,...,...,...,...
8912,fff0f8253670e9b7144db69078ab99b9,2020-02-26 16:55:44,10,20,15567.0
8913,fff33f0533feff490e5437a801b9098e,2020-01-26 21:16:07,0,2,18054.0
8914,fffafea985efe469192a90d253823990,2020-03-05 13:37:45,7,20,9431.0
8915,fffb6ec5e50ca3014c0aefef4f3a95d4,2020-01-24 01:45:19,2,9,36805.0


In [184]:
import pandas as pd
from pandasql import sqldf

data = pd.read_csv("raw_event_log.csv")
new_dataframe = pd.DataFrame(data)

pysqldf = lambda q: sqldf(q, globals())

q="""SELECT  strftime("%m-%Y", d1.timestamp) as 'Activation Month', Count(d1.event) as Total, SUM(event='login')/6 AS Average_login
    FROM new_dataframe d1
    
   group by strftime("%m-%Y", d1.timestamp)
;"""

result = pysqldf(q)
result

result.to_csv("Database2.csv", encoding='utf-8', index=False)

,Activation Month,Total,Average_login
0,01-2020,44031,1218
1,02-2020,55733,1846
2,03-2020,69540,2260
3,04-2020,50356,1868
4,05-2020,6851,282
5,06-2020,85,3
